In [ ]:
from pymongo import MongoClient

from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from textblob import Word
import nltk
import sklearn
import pandas as pd
import numpy as np
import re
import pprint

%pylab inline

pd.options.display.max_columns = 999

In [ ]:
client = MongoClient()

scotus_db = client.scotus_db
scotus = scotus_db.scotus_collection

### Text Cleaning and Set Up


In [ ]:
#Functions for Text Analysis
def sentiment_analysis(x):
    return TextBlob(x).sentiment
def polarity_analysis(x):
    return TextBlob(x).polarity
def subjectivity_analysis(x):
    return TextBlob(x).subjectivity

In [ ]:
#Functions for Topics
from sklearn import decomposition
from nltk.stem import WordNetLemmatizer 


def lemmatize_text(text):
    '''
    **Use like this**
    df['lem_text'] = df['text'].apply(lemmatize_text)
    '''
    lemm = WordNetLemmatizer()
    new_text = ''
    for w in nltk.word_tokenize(text): 
        new_text += lemm.lemmatize(w) + ' '
    return new_text

def get_topics_cv(df):
    count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(
        ngram_range=(1, 2),
        stop_words='english',
        token_pattern="\\b[a-z][a-z][a-z]+\\b",
        min_df=1
    )
    count_vectorizer.fit(df)
    # Create the term-document matrix
    counts = count_vectorizer.transform(df)
    # this gives us a [num_documents, num_features] sparse matrix
    print(f"Shape: {counts.shape}")
    n_topics = 10
    lda = decomposition.LatentDirichletAllocation\
                        (n_components=n_topics, learning_method="online"
                        , max_iter=5, n_jobs=-1)

    lda.fit(counts)
    print("**Topics**")
    print_top_words(lda, count_vectorizer.get_feature_names(), 15)
    
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % (topic_idx+1)
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
def topics_by_year(year):
    
    cursor_func = scotus.find({"term": str(year)})
    df_func = pd.DataFrame(list(cursor_func))

    df_func['lem_text'] = df_func['text'].apply(lemmatize_text)
    df_func_2=df_func['lem_text'].copy()
    print(f"SCOTUS Topics for Year: {year}")
    get_topics(df_func_2)

I will run sentiment analysis on all cases in **df**

In [ ]:
cursor = scotus.find\
    "term":{"$in":['2006','2007','2008','2009',\
                               '2010','2011','2012','2013',\
                              '2014','2015','2016','2017']}
          
                   
df = pd.DataFrame(list(cursor))

In [ ]:
df['tb_polarity'] = df['text'].apply(polarity_analysis) #this takes a while
print("polarity done")
df['tb_subjectivity'] = df['text'].apply(subjectivity_analysis)#this takes a while
print("subjectivity done")

In [ ]:
agg_funcs = {"lem_text":'sum', 'tb_subjectivity':'sum', "tb_polarity":'sum'}

df['lem_text']=df['text'].apply(lemmatize_text)
df['case_term'] = df['case_name']+ ' ' + df['term']

df = df.groupby(['case_term']).agg(agg_funcs).reset_index()
case_term = list(df['case_term'])

Now I'm setting up another dataframe **df2** that will have the voting results by justice (who spoke during the case). We'll then join df2 (voting results) with df (text analysis)

In [ ]:
df2 = pd.DataFrame(list(cursor))

In [ ]:
df_2['lem_text']=df_2['text'].apply(lemmatize_text)
df_2['case_term'] = df_2['case_name']+ ' ' + df_2['term']

df_groupby = df_2.groupby(['case_term','speaker','voting_result']).max()

df_groupby= df_groupby.reset_index()
df_groupby = df_groupby[['case_term','speaker','voting_result']]

df_pivot = df_groupby.pivot(index='case_term',columns='speaker',values='voting_result')

df_votes_cases = df.merge(df_pivot,left_on='case_term',right_index=True)

pd.to_pickle(df_votes_cases,'2006_2017_votes_AND_cases')
df_votes_cases.sample(2)


![Sample](df_votes_cases_sample.png)

In [ ]:
df = df_votes_cases

In [ ]:
def dict_topic_words(model, feature_names, n_top_words):
    topic_dict={}
    for topic_idx, topic in enumerate(model.components_):
        message = ""
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        topic_dict[topic_idx]=message
    return topic_dict

dict_topwords = dict_topic_words(lda,count_vectorizer.get_feature_names(),5)

# now let's transform our documents to topic-space
print(f"shape before transforming to topic space: {counts.shape}")

doc_topics = lda.transform(counts)

print(f"shape after transforming to topic space: {doc_topics.shape}")

#Printed - pasted below
# shape before transforming to topic space: (827, 15081)
# shape after transforming to topic space: (827, 30)

### Topic Analysis - Count Vectorizer into LDA
